In [ ]:
#!/usr/bin/env python
# Given a .docx file, extract the table and save it as a .csv file
# This is version 1.0 of the script
# Modified script from: https://carstenknoch.com/2020/02/qualitative-data-analysis-with-microsoft-word-comments-python-updated/
# Date started: 13.07.2021

# https://medium.com/@karthikeyan.eaganathan/read-tables-from-docx-file-to-pandas-dataframes-f7e409401370

In [ ]:
# Original code
#!/usr/bin/env python
# Given a .docx file, extract a CSV list of all tagged (commented) text
# This is version 6.0 of the script
# Date: 12 February 2020

import zipfile
import csv
from bs4 import BeautifulSoup as Soup
import tkinter as tk
from tkinter import filedialog
import re

# Show file selection dialog box
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()
root.update()

with open('/'.join(paths[0].split('/')[0:-1])+'/output.csv', 'w', newline='', encoding='utf-8-sig') as f:
	csvw = csv.writer(f)
	# loop through each selected file
	for path in paths:
		# Write a header line with the filename
		csvw.writerow([path.split('/')[-1], ''])
		# .docx files are really ZIP files with a separate 'file' within them for the document
		# itself and the text of the comments. This unzips the file and parses the comments.xml
		# file within it, which contains the comment (label) text
		unzip = zipfile.ZipFile(path)
		comments = Soup(unzip.read('word/comments.xml'), 'lxml')
		# The structure of the document itself is more complex and we need to do some
		# preprocessing to handle multi-paragraph and nested comments, so we unzip
		# it into a string first
		doc = unzip.read('word/document.xml').decode()
		# Find all the comment start and end locations and store them in dictionaries
		# keyed on the unique ID for each comment
		start_loc = {x.group(1): x.start() for x in re.finditer(r'<w:commentRangeStart.*?w:id="(.*?)"', doc)}
		end_loc = {x.group(1): x.end() for x in re.finditer(r'<w:commentRangeEnd.*?w:id="(.*?)".*?>', doc)}
		# loop through all the comments in the comments.xml file
		for c in comments.find_all('w:comment'):
			c_id = c.attrs['w:id']
			# Use the locations we found earlier to extract the xml fragment from the document for
			# each comment ID, adding spaces to separate any paragraphs in multi-paragraph comments
			xml = re.sub(r'(<w:p .*?>)', r'\1 ', doc[start_loc[c_id]:end_loc[c_id] + 1])
			# Parse the XML fragment, extract any text and write to file along with the label text
			csvw.writerow([''.join(c.findAll(text=True)), ''.join(Soup(xml, 'lxml').findAll(text=True))])
		unzip.close()

In [ ]:
import zipfile
import csv
from bs4 import BeautifulSoup as Soup

In [2]:
import tkinter as tk
from tkinter import filedialog
import re
import os
import sys
import docx
from docx import Document
import pandas as pd

In [ ]:
# Show file selection dialog box
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()
root.update()

with open('/'.join(paths[0].split('/')[0:-1])+'/output.csv', 'w', newline='', encoding='utf-8-sig') as f:
	csvw = csv.writer(f)
	# loop through each selected file
	for path in paths:
		# Write a header line with the filename
		csvw.writerow([path.split('/')[-1], ''])
		# .docx files are really ZIP files with a separate 'file' within them for the document
		# itself and the text of the comments. This unzips the file and parses the comments.xml
		# file within it, which contains the comment (label) text
		unzip = zipfile.ZipFile(path)
        unzip.close()

In [123]:
#latest version

def read_docx_table(**kwarg):
    root = tk.Tk()
    root.withdraw()
    paths = filedialog.askopenfilename()
    root.update()
    document = Document(paths)
    table_num = int(input("Please input table number, index starts from 0: "))
    nheader = int(input("Please input number of headers: "))
    
    table = document.tables[table_num-1]
    data =  [[cell.text for cell in row.cells] for row in table.rows]
    df = pd.DataFrame(data)
    
    if nheader == 1:
        df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
    elif nheader == 2:
        outside_col, inside_col = df.iloc[0], df.iloc[1]
        hier_index = pd.MultiIndex.from_tuples(list(zip(outside_col, inside_col)))
        df = df.DataFrame(data, columns = hier_index).drop(df.index[[0,1]]).reset_index(drop=True)
    elif nheader > 2:
        print("More than two headers not currently supported")
        df = pd.DataFrame()
    return df
    document.close()
    
#     df.index=[0]*len(df)
#     df2 = df.pivot(index=None, columns="Question", values="Answer")
#     df2.to_excel(filedialog.asksaveasfilename())
        
read_docx_table()

Please input table number, index starts from 0:  0
Please input number of headers:  1


,Question,Answer
0,Partner name?,Partner 1
1,Are you an Iwi owned provider?,Yen
2,Do you have any of the following services?,
3,"Primary Care (including: Medical Centres, Afte...",Service_PC_1\nService_PC_2\nService _PC_3
4,Public Health (these is a high priority for Mā...,Service_PH_1
5,Mental Health,Service_MH_1
6,Community Services and Kaupapa Māori Services ...,Service_CSKPM_1\nService_CSKPM_2
7,Disabilities,
8,"COVID-19 - Swabbing Centres, Contract Tracing,...",Service_C19_1\nService_C19_2
9,COVID-19 - Vaccination Centres and Vaccination...,


In [124]:
df
#read_docx_table()
# C:\Users\stefan.lim\Desktop\Template_ex1.docx
# C:\Users\stefan.lim\Downloads\Te_Pūtea_Whakatupu_-_Te_Kete_Aronui\Working files\TKA MUMA Report Q2 2021(commented).docx

NameError: name 'df' is not defined

In [102]:
df.index=[0]*len(df)
df2 = df.pivot(index=None, columns="Question", values="Answer")

NameError: name 'df' is not defined

In [17]:
result = pd.concat([df2,df4])
result.to_excel('testconcat.xlsx')

In [15]:
df3.index=[0]*len(df3)
df4 = df3.pivot(index=None, columns="Question", values="Answer")
df4

Question,Are you an Iwi owned provider?,"COVID-19 - Swabbing Centres, Contract Tracing, Communications",COVID-19 - Vaccination Centres and Vaccination roll-out,"Community Services and Kaupapa Māori Services (including District Health Nurses, Visual Hearing Test Nurses, Before School Checks, Outreach Immunisations, Pre-natal and Post-natal care)",Disabilities,Do you have any of the following services?,How many registered whānau do you have? How many are Māori?,Mental Health,Partner name?,"Primary Care (including: Medical Centres, After Hours, Dentists, Audiology, Radiology, Physiotherapy, Pharmacy, Home Based Support)",Public Health (these is a high priority for Māori Health as it is preventative focussed).,Technology (including Digital Health and Tele-Health),"What patient management system do you use? (Medtech 32 or Evolution, Indici, MyPractice, other)"
0,No,Service_C19_1\nService_C19_2,,Service_CSKPM_1\nService_CSKPM_2,,,9001,Service_MH_1,Partner 2,Service_PC_1\nService_PC_2\nService _PC_3,Service_PH_1,Service_IT_1,Medtech_32


In [25]:
result = result.set_index('Partner name?').reset_index()
result.to_excel('testconcatreindex.xlsx')

In [91]:
df.str.split('\n', expand=True)

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
        unzip = zipfile.ZipFile(path)
        comments = Soup(unzip.read('word/comments.xml'), 'lxml')
		# The structure of the document itself is more complex and we need to do some
		# preprocessing to handle multi-paragraph and nested comments, so we unzip
		# it into a string first
		doc = unzip.read('word/document.xml').decode()
		# Find all the comment start and end locations and store them in dictionaries
		# keyed on the unique ID for each comment
		start_loc = {x.group(1): x.start() for x in re.finditer(r'<w:commentRangeStart.*?w:id="(.*?)"', doc)}
		end_loc = {x.group(1): x.end() for x in re.finditer(r'<w:commentRangeEnd.*?w:id="(.*?)".*?>', doc)}
		# loop through all the comments in the comments.xml file
		for c in comments.find_all('w:comment'):
			c_id = c.attrs['w:id']
			# Use the locations we found earlier to extract the xml fragment from the document for
			# each comment ID, adding spaces to separate any paragraphs in multi-paragraph comments
			xml = re.sub(r'(<w:p .*?>)', r'\1 ', doc[start_loc[c_id]:end_loc[c_id] + 1])
			# Parse the XML fragment, extract any text and write to file along with the label text
			csvw.writerow([''.join(c.findAll(text=True)), ''.join(Soup(xml, 'lxml').findAll(text=True))])
		unzip.close()